In [149]:
from PlayerScraper import PlayerScraper
import pandas as pd
from io import StringIO
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import time
import pickle

In [150]:
ps = PlayerScraper()

In [29]:
ps.driver.get('https://www.basketball-reference.com/players/j/jaricma01.html')

source = ps.driver.page_source

In [30]:
def get_dataoverheader(row):
        try:
            return row.attrs['data-over-header']
        except:
            pass
        
        
def gettext(row):
    return row.getText()
        
def mapgettext(row, gettext = gettext):
    return list(map(gettext, row))

def get_tds(row):
    return row.find_all('td')

In [31]:
def get_table_stats(tbl_html):

    suffixs = list(map(get_dataoverheader, tbl_html.find("thead").find_all('th',
                                                 ["poptip sort_default_asc center", 
                                                  "poptip center"])))
    columns = list(map(gettext,
                    tbl_html.find("thead").find_all('th',
                                                 ["poptip sort_default_asc center", 
                                                  "poptip center"])))
    
    columnsWsuffixs = [f'{columns[i]}' if (suffixs[i] == None) or (suffixs[i] == '\xa0') else f'{columns[i]}_{suffixs[i]}' for i in range(len(columns)) ]

    
#     columnsWsuffixs_filtered = [col for col in columnsWsuffixs if (col != '\xa0')]
#     columnsWsuffixs_filtered = list(filter(None, columnsWsuffixs_filtered))
    
    seasons = list(map(gettext,tbl_html.find("tbody").find_all("th")))
    
    tbl_cells = list(map(get_tds,tbl_html.find("tbody").find_all("tr")))
    

    table_data = list(map(mapgettext, tbl_cells))
    elems_to_delete = []
    for i in range(len(table_data)):
        if len(table_data[i]) != (len(columns)-1):
            elems_to_delete.append(i)     

    if elems_to_delete:
        for elem in sorted(elems_to_delete, reverse = True):
            del table_data[elem]
        


    df = pd.DataFrame(data = table_data, 
             index = seasons).reset_index().replace('', np.nan)



    df.columns = columnsWsuffixs
    
    return df

In [32]:
def get_all_tables(page):
    soup = BeautifulSoup(page)
    all_tbls = soup.find_all(class_ = ["row_summable sortable stats_table now_sortable",
                                "row_summable sortable stats_table now_sortable sticky_table re1 le1"])
    tables_to_scrape = dict.fromkeys([tbl.attrs['id'] for tbl in all_tbls])
    
    all_data = [get_table_stats(all_tbls[i]) for i in range(len(all_tbls))]
    
    dfs = {all_tbls[i].attrs['id']: all_data[i] for i in range(len(all_tbls))}    
    
    return dfs

In [165]:
# import string
# letters = [letter for letter in string.ascii_lowercase]

# player_url = 'https://www.basketball-reference.com'

# player_url_letters = {f'{player_url}/players/{x}':None for x in letters}

# def get_links(row):
#     try:
#         return row.find('th').find('a').attrs['href']
#     except:
#         pass    

# import time

# for i in player_url_letters:
#     ps.driver.get(i)
#     time.sleep(2)
#     player_url_letters[i] = ps.driver.page_source

# page_sources = list(player_url_letters.values())

In [ ]:
def get_player_urls_from_page(page_html):
    
    soup = BeautifulSoup(page_html)
    columns = list(map(gettext,
                    soup.find("thead").find_all('th',
                                                 ["poptip sort_default_asc center", 
                                                  "poptip center"])))

    player_names = list(map(gettext,
                            soup.find(class_ = ['sortable stats_table now_sortable',
                                               'sortable stats_table now_sortable sticky_table re1 le1']).\
                            find("tbody").find_all("th")))
    
    player_names = list(filter(lambda x: x not in columns, player_names))
    
    tbl_cells = list(map(get_tds,
                    soup.find(class_ = ['sortable stats_table now_sortable',
                                               'sortable stats_table now_sortable sticky_table re1 le1']).\
                         find("tbody").find_all("tr")
                    ))
    
    table_data = np.array(list(filter(None,
                                      (map(mapgettext, 
                                    tbl_cells))
                                     )
                              )
                         )

    
    player_links = list(filter(None,map(get_links,
         soup.find(class_ = ['sortable stats_table now_sortable',
                                               'sortable stats_table now_sortable sticky_table re1 le1']).\
                            find("tbody").find_all("tr")
        )))

    df = pd.DataFrame(data = table_data,
                     index = player_names).reset_index()
    try:
        df.columns = columns
        df['Player_links'] = player_links
    except:
        pass
    return df

player_df = pd.concat(list(map(get_player_urls_from_page, 
                  page_sources))).dropna(axis = 1).reset_index(drop=True)

player_df['Player_links'] = [f'{player_url}{x}' for x in player_df.Player_links.values]

# list(map(lambda x: f'{player_url}{x}',
#          player_df.Player_links.values
#                                      ))

player_df['Player_links'][0]

def get_player_bbrefUrlID(url):
    return re.findall("([^/]+$)", url)[0][:-5]

player_df['BBrefID'] = list(map(get_player_bbrefUrlID,
                               player_df.Player_links.values))

player_df.to_csv('all_players.csv', index = False)

In [173]:
player_df

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges,Player_links,BBrefID
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke,https://www.basketball-reference.com/players/a...,abdelal01
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State,https://www.basketball-reference.com/players/a...,abdulza01
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA,https://www.basketball-reference.com/players/a...,abdulka01
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,https://www.basketball-reference.com/players/a...,abdulma02
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State",https://www.basketball-reference.com/players/a...,abdulta01
...,...,...,...,...,...,...,...,...,...,...
4798,Ante Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",NaN,https://www.basketball-reference.com/players/z...,zizican01
4799,Jim Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,https://www.basketball-reference.com/players/z...,zoetji01
4800,Bill Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne,https://www.basketball-reference.com/players/z...,zopfbi01
4801,Ivica Zubac,2017,2020,C,7-0,240.0,"March 18, 1997",NaN,https://www.basketball-reference.com/players/z...,zubaciv01


In [ ]:
#get this all into the playercraper class and then figure out how to use AWS postgres to host this shit

In [38]:
player_df = pd.read_csv('all_players.csv')

In [41]:
player_df['From'] = pd.to_numeric(player_df['From'])

In [43]:
players_from_lebron_onwards = player_df[player_df['From'] > 2003]

In [45]:
player_links_dict_with_page_source = dict.fromkeys(players_from_lebron_onwards['Player_links'].values)

In [48]:
for i in player_links_dict_with_page_source:
    ps.driver.get(i)
    time.sleep(1)
    player_links_dict_with_page_source[i] = ps.driver.page_source

In [50]:
keys = list(player_links_dict_with_page_source.keys())

In [172]:
get_all_tables(player_links_dict_with_page_source[keys[0]])['totals']

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2016-17,23,OKC,NBA,SG,68,6,1055,134,341,...,.898,18,68,86,40,37,8,33,114,406
1,2017-18,24,OKC,NBA,SG,75,8,1134,115,291,...,.848,26,88,114,28,38,8,25,124,353
2,2018-19,25,OKC,NBA,SG,31,2,588,56,157,...,.923,5,43,48,20,17,6,14,53,165


In [174]:
player_tables=[]

for i in range(len(keys)):
    player_tables.append(get_all_tables(player_links_dict_with_page_source[keys[i]]))

In [175]:
len(player_tables)

1376

In [176]:
len(keys)

1376

In [177]:
player_tables[0]

{'per_game':     Season Age   Tm   Lg Pos   G GS    MP   FG  FGA  ...   FT%  ORB  DRB  TRB  \
 0  2016-17  23  OKC  NBA  SG  68  6  15.5  2.0  5.0  ...  .898  0.3  1.0  1.3   
 1  2017-18  24  OKC  NBA  SG  75  8  15.1  1.5  3.9  ...  .848  0.3  1.2  1.5   
 2  2018-19  25  OKC  NBA  SG  31  2  19.0  1.8  5.1  ...  .923  0.2  1.4  1.5   
 
    AST  STL  BLK  TOV   PF  PTS  
 0  0.6  0.5  0.1  0.5  1.7  6.0  
 1  0.4  0.5  0.1  0.3  1.7  4.7  
 2  0.6  0.5  0.2  0.5  1.7  5.3  
 
 [3 rows x 30 columns],
 'totals':     Season Age   Tm   Lg Pos   G GS    MP   FG  FGA  ...   FT% ORB DRB  TRB  \
 0  2016-17  23  OKC  NBA  SG  68  6  1055  134  341  ...  .898  18  68   86   
 1  2017-18  24  OKC  NBA  SG  75  8  1134  115  291  ...  .848  26  88  114   
 2  2018-19  25  OKC  NBA  SG  31  2   588   56  157  ...  .923   5  43   48   
 
   AST STL BLK TOV   PF  PTS  
 0  40  37   8  33  114  406  
 1  28  38   8  25  124  353  
 2  20  17   6  14   53  165  
 
 [3 rows x 30 columns],
 'per_minu

In [74]:
max_tbl = 0
max_keys = []
for i in range(len(player_tables)):
    if len(player_tables[i]) > max_tbl:
        max_tbl=len(player_tables[i])
        max_keys = list(player_tables[i].keys())

In [86]:
def generate_gets(max_keys):
    for i in range(len(max_keys)):
        print(f'''
def get_{max_keys[i]}(vals):
    try:
        return vals['{max_keys[i]}']
    except:
        pass''')
        print('---------------------')

In [88]:
def get_per_game(vals):
    try:
        return vals['per_game']
    except:
        pass
    
def get_totals(vals):
    try:
        return vals['totals']
    except:
        pass
    
def get_per_minute(vals):
    try:
        return vals['per_minute']
    except:
        pass
    
def get_per_poss(vals):
    try:
        return vals['per_poss']
    except:
        pass
    
def get_advanced(vals):
    try:
        return vals['advanced']
    except:
        pass
    

def get_adj_shooting(vals):
    try:
        return vals['adj-shooting']
    except:
        pass
    

def get_shooting(vals):
    try:
        return vals['shooting']
    except:
        pass
    

def get_pbp(vals):
    try:
        return vals['pbp']
    except:
        pass


def get_playoffs_per_game(vals):
    try:
        return vals['playoffs_per_game']
    except:
        pass


def get_playoffs_totals(vals):
    try:
        return vals['playoffs_totals']
    except:
        pass


def get_playoffs_per_minute(vals):
    try:
        return vals['playoffs_per_minute']
    except:
        pass


def get_playoffs_per_poss(vals):
    try:
        return vals['playoffs_per_poss']
    except:
        pass


def get_playoffs_advanced(vals):
    try:
        return vals['playoffs_advanced']
    except:
        pass


def get_playoffs_shooting(vals):
    try:
        return vals['playoffs_shooting']
    except:
        pass


def get_playoffs_pbp(vals):
    try:
        return vals['playoffs_pbp']
    except:
        pass


def get_all_star(vals):
    try:
        return vals['all_star']
    except:
        pass


In [157]:
def get_tables(vals, tbl_id):
    try:
        return vals[tbl_id]
    except:
        pass

In [178]:
per_game = pd.concat(list(map(get_per_game, player_tables)))



# per_game = pd.concat(list(map(get_tables(tbl_id='per_game'), player_tables)))

In [179]:
per_game

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2016-17,23,OKC,NBA,SG,68,6,15.5,2.0,5.0,...,.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0
1,2017-18,24,OKC,NBA,SG,75,8,15.1,1.5,3.9,...,.848,0.3,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7
2,2018-19,25,OKC,NBA,SG,31,2,19.0,1.8,5.1,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
0,2005-06,23,DET,NBA,SG,5,0,7.0,0.8,3.2,...,NaN,0.2,0.8,1.0,0.8,0.2,0.0,0.8,0.8,1.8
1,2008-09,26,TOT,NBA,SG,25,0,8.0,1.2,3.0,...,.500,0.3,0.6,1.0,0.5,0.2,0.2,0.3,0.4,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2017-18,20,LAL,NBA,C,43,0,9.5,1.4,2.8,...,.765,1.0,1.8,2.9,0.6,0.2,0.3,0.6,1.1,3.7
2,2018-19,21,TOT,NBA,C,59,37,17.6,3.6,6.4,...,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9
3,2018-19,21,LAL,NBA,C,33,12,15.6,3.4,5.8,...,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5
4,2018-19,21,LAC,NBA,C,26,25,20.2,3.8,7.2,...,.733,2.3,5.3,7.7,1.5,0.4,0.9,1.4,2.5,9.4


In [167]:
per_game = pd.concat([get_tables(p, tbl_id='per_game') for p in player_tables])

In [171]:
per_game

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_link
0,2016-17,23,OKC,NBA,SG,68,6,15.5,2.0,5.0,...,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,https://www.basketball-reference.com/players/a...
1,2017-18,24,OKC,NBA,SG,75,8,15.1,1.5,3.9,...,0.3,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7,https://www.basketball-reference.com/players/a...
2,2018-19,25,OKC,NBA,SG,31,2,19.0,1.8,5.1,...,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3,https://www.basketball-reference.com/players/a...
0,2005-06,23,DET,NBA,SG,5,0,7.0,0.8,3.2,...,0.2,0.8,1.0,0.8,0.2,0.0,0.8,0.8,1.8,https://www.basketball-reference.com/players/a...
1,2008-09,26,TOT,NBA,SG,25,0,8.0,1.2,3.0,...,0.3,0.6,1.0,0.5,0.2,0.2,0.3,0.4,2.9,https://www.basketball-reference.com/players/a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2017-18,20,LAL,NBA,C,43,0,9.5,1.4,2.8,...,1.0,1.8,2.9,0.6,0.2,0.3,0.6,1.1,3.7,https://www.basketball-reference.com/players/z...
2,2018-19,21,TOT,NBA,C,59,37,17.6,3.6,6.4,...,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9,https://www.basketball-reference.com/players/z...
3,2018-19,21,LAL,NBA,C,33,12,15.6,3.4,5.8,...,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5,https://www.basketball-reference.com/players/z...
4,2018-19,21,LAC,NBA,C,26,25,20.2,3.8,7.2,...,2.3,5.3,7.7,1.5,0.4,0.9,1.4,2.5,9.4,https://www.basketball-reference.com/players/z...


In [91]:
per_game = pd.concat(list(map(get_per_game, player_tables)))


totals = pd.concat(list(map(get_totals, player_tables)))


per_minute = pd.concat(list(map(get_per_minute, player_tables)))


per_poss = pd.concat(list(map(get_per_poss, player_tables)))


per_advanced = pd.concat(list(map(get_advanced, player_tables)))

In [95]:
per_game.to_csv('per_game.csv', index = False)
totals.to_csv('totals.csv', index = False)
per_minute.to_csv('per_minute.csv', index = False)
per_poss.to_csv('per_poss.csv', index = False)
per_advanced.to_csv('per_advanced.csv', index = False)

In [92]:
per_shooting = pd.concat(list(map(get_shooting, player_tables)))


per_pbp = pd.concat(list(map(get_pbp, player_tables)))


playoffs_per_game = pd.concat(list(map(get_playoffs_per_game, player_tables)))


playoff_totals = pd.concat(list(map(get_playoffs_totals, player_tables)))

In [96]:
per_shooting.to_csv('per_shooting.csv', index = False)
per_pbp.to_csv('per_pbp.csv', index = False)
playoffs_per_game.to_csv('playoffs_per_game.csv', index = False)
playoff_totals.to_csv('playoff_totals.csv', index = False)

In [93]:
playoffs_per_minute = pd.concat(list(map(get_playoffs_per_minute, player_tables)))


playoffs_per_poss = pd.concat(list(map(get_playoffs_per_poss, player_tables)))


playoffs_per_advanced = pd.concat(list(map(get_playoffs_advanced, player_tables)))


all_star = pd.concat(list(map(get_all_star, player_tables)))


playoffs_per_shooting = pd.concat(list(map(get_playoffs_shooting, player_tables)))


playoffs_per_pbp = pd.concat(list(map(get_playoffs_pbp, player_tables)))

In [97]:
playoffs_per_minute.to_csv('playoffs_per_minute.csv', index=False)
playoffs_per_poss.to_csv('playoffs_per_poss.csv', index=False)
playoffs_per_advanced.to_csv('playoffs_per_advanced.csv', index=False)
all_star.to_csv('all_star.csv', index=False)
playoffs_per_shooting.to_csv('playoffs_per_shooting.csv', index=False)
playoffs_per_pbp.to_csv('playoffs_per_pbp.csv', index=False)

In [121]:
t[0].columns[16:]

Index([' ', 'FG_League Shooting %', '2P_League Shooting %',
       '3P_League Shooting %', 'eFG_League Shooting %', 'FT_League Shooting %',
       'TS_League Shooting %', 'FTr_League Shooting %',
       '3PAr_League Shooting %', ' ', 'FG+_League-Adjusted',
       '2P+_League-Adjusted', '3P+_League-Adjusted', 'eFG+_League-Adjusted',
       'FT+_League-Adjusted', 'TS+_League-Adjusted', 'FTr+_League-Adjusted',
       '3PAr+_League-Adjusted', ' ', 'FG Add', 'TS Add', 'Player_link'],
      dtype='object')

In [111]:
adj_shooting_player_columns = [
    'FG_player Shooting %', '2P_player Shooting %',
       '3P_player Shooting %', 'eFG_player Shooting %',
       'FT_player Shooting %', 'TS_player Shooting %',
       'FTr_player Shooting %', '3PAr_player Shooting %'
]

In [117]:
t[0].columns[:8]

Index(['Season', 'Age', 'Team', 'Lg', 'Pos', 'G', 'MP', ' '], dtype='object')

In [122]:
adj_shooting_columns = list(t[0].columns[:8]) + [
    'FG_player Shooting %', '2P_player Shooting %',
       '3P_player Shooting %', 'eFG_player Shooting %',
       'FT_player Shooting %', 'TS_player Shooting %',
       'FTr_player Shooting %', '3PAr_player Shooting %'
] + list(t[0].columns[16:])

In [156]:
adj_shooting_columns


['Season',
 'Age',
 'Team',
 'Lg',
 'Pos',
 'G',
 'MP',
 '\xa0',
 'FG_player Shooting %',
 '2P_player Shooting %',
 '3P_player Shooting %',
 'eFG_player Shooting %',
 'FT_player Shooting %',
 'TS_player Shooting %',
 'FTr_player Shooting %',
 '3PAr_player Shooting %',
 '\xa0',
 'FG_League Shooting %',
 '2P_League Shooting %',
 '3P_League Shooting %',
 'eFG_League Shooting %',
 'FT_League Shooting %',
 'TS_League Shooting %',
 'FTr_League Shooting %',
 '3PAr_League Shooting %',
 '\xa0',
 'FG+_League-Adjusted',
 '2P+_League-Adjusted',
 '3P+_League-Adjusted',
 'eFG+_League-Adjusted',
 'FT+_League-Adjusted',
 'TS+_League-Adjusted',
 'FTr+_League-Adjusted',
 '3PAr+_League-Adjusted',
 '\xa0',
 'FG Add',
 'TS Add',
 'Player_link']

In [114]:
adj_shooting_list = list(map(get_adj_shooting, player_tables))

In [130]:
index_of_nonetypes = []

In [131]:
for i in range(len(adj_shooting_list)):
    if adj_shooting_list[i] is not None:
        adj_shooting_list[i].columns = adj_shooting_columns
    else:
        index_of_nonetypes.append(i)

In [132]:
index_of_nonetypes

[178]

In [133]:
del adj_shooting_list[index_of_nonetypes[0]]

In [135]:
per_adj_shooting = pd.concat(adj_shooting_list)
per_adj_shooting.to_csv('per_adj_shooting.csv', index = False)

In [153]:
list(per_adj_shooting.columns)

['Season',
 'Age',
 'Team',
 'Lg',
 'Pos',
 'G',
 'MP',
 '\xa0',
 'FG_player Shooting %',
 '2P_player Shooting %',
 '3P_player Shooting %',
 'eFG_player Shooting %',
 'FT_player Shooting %',
 'TS_player Shooting %',
 'FTr_player Shooting %',
 '3PAr_player Shooting %',
 '\xa0',
 'FG_League Shooting %',
 '2P_League Shooting %',
 '3P_League Shooting %',
 'eFG_League Shooting %',
 'FT_League Shooting %',
 'TS_League Shooting %',
 'FTr_League Shooting %',
 '3PAr_League Shooting %',
 '\xa0',
 'FG+_League-Adjusted',
 '2P+_League-Adjusted',
 '3P+_League-Adjusted',
 'eFG+_League-Adjusted',
 'FT+_League-Adjusted',
 'TS+_League-Adjusted',
 'FTr+_League-Adjusted',
 '3PAr+_League-Adjusted',
 '\xa0',
 'FG Add',
 'TS Add',
 'Player_link']

In [98]:
# keys

In [142]:
import matplotlib.pyplot as plt

%matplotlib inline

In [148]:
per_adj_shooting

,Season,Age,Team,Lg,Pos,G,MP,,FG_player Shooting %,2P_player Shooting %,...,3P+_League-Adjusted,eFG+_League-Adjusted,FT+_League-Adjusted,TS+_League-Adjusted,FTr+_League-Adjusted,3PAr+_League-Adjusted,,FG Add,TS Add,Player_link
0,2016-17,23,OKC,NBA,SG,68,1055,NaN,.393,.425,...,106,103,116,101,53,229,NaN,11.7,5.5,https://www.basketball-reference.com/players/a...
1,2017-18,24,OKC,NBA,SG,75,1134,NaN,.395,.443,...,105,104,111,102,63,225,NaN,10.7,6.8,https://www.basketball-reference.com/players/a...
2,2018-19,25,OKC,NBA,SG,31,588,NaN,.357,.500,...,91,93,120,91,32,225,NaN,-11.6,-17.1,https://www.basketball-reference.com/players/a...
0,2005-06,23,DET,NBA,SG,5,35,NaN,.250,.273,...,56,57,NaN,53,NaN,154,NaN,-6.7,-8.1,https://www.basketball-reference.com/players/a...
1,2008-09,26,TOT,NBA,SG,25,199,NaN,.395,.411,...,95,88,65,82,43,117,NaN,-9.0,-15.5,https://www.basketball-reference.com/players/a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2017-18,20,LAL,NBA,C,43,410,NaN,.500,.504,...,NaN,96,100,100,166,2,NaN,-5.2,.3,https://www.basketball-reference.com/players/z...
2,2018-19,21,TOT,NBA,C,59,1040,NaN,.559,.559,...,NaN,107,105,108,129,NaN,NaN,26.7,38.8,https://www.basketball-reference.com/players/z...
3,2018-19,21,LAL,NBA,C,33,516,NaN,.580,.580,...,NaN,111,113,113,132,NaN,NaN,21.7,32.5,https://www.basketball-reference.com/players/z...
4,2018-19,21,LAC,NBA,C,26,524,NaN,.538,.538,...,NaN,103,96,103,125,NaN,NaN,5.0,6.3,https://www.basketball-reference.com/players/z...


In [67]:
for i in range(len(player_tables)):
    for key in player_tables[i]:
        player_tables[i][key]['Player_link'] = keys[i]